# Social Media Relationships with Ordinary Least Squares

**Linear models** in the form of **ordinary least squares regression** can often be help with understanding relationships between variables. In this exercise you will analyze synthetic data which relates game players' characteristics to social media engagement. 

To start, execute the code in the cell below to import the packages you will need for this example. 

In [8]:
import numpy as np
import numpy.random as nr
from scipy.stats import truncnorm
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd

## matplotlib with display of graphs inline
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

The code in the cell below generates synthetic data with characteristics of game players and social media engagement. These variables are drawn from a truncated multivariate Normal distribution, with limits of 0.0 and 10.0. Execute this code.

In [9]:
## Create data set as multivariate Normal
covariance = np.array([[1.0,0.7,0.7,0.0],
                      [0.7,1.0,0.6,0.7],
                      [0.7,0.6,1.0,0.1],
                      [0.0,0.7,0.1,1.0]])
effect_data = pd.DataFrame(nr.multivariate_normal(mean=[3.0,2.0,2.0,2.0], cov=covariance, size=500),
                           columns=['Fan','TimePlaying','SocialMedia','GameFamiliarity'])

## Truncate values to range 0.0 <= x <= 10.0
effect_data[effect_data < 0.0] = 0.0
effect_data[effect_data > 10.0] = 10.0

The code in the cell below transforms the TimePlaying variable by squaring the values. This transformation gives the transformed variable a positive skew. The code also rounds the values to two digits. Execute this code and examine the resulting data frame. 

In [10]:
## Square the values of TimePlaying to give positive skew
effect_data['TimePlaying'] = np.square(effect_data['TimePlaying'])

## Standardize the dependent variables
dependent_cols = ['Fan','TimePlaying','GameFamiliarity']
for col in dependent_cols: 
    effect_data[col] = (effect_data[col] - effect_data[col].mean()).divide(effect_data[col].var())

## And round all values to 2 decimal places
effect_data = effect_data.round(decimals=2)
effect_data

,Fan,TimePlaying,SocialMedia,GameFamiliarity
0,0.01,0.25,1.88,1.66
1,1.32,0.23,2.87,0.03
2,0.07,0.13,1.83,1.00
3,-0.47,0.12,1.55,1.18
4,-1.12,-0.23,0.60,-0.05
5,0.88,0.08,3.32,0.02
6,1.27,0.47,4.20,0.76
7,0.75,0.50,1.11,1.75
8,0.14,0.02,0.83,0.78
9,-0.17,0.03,1.24,0.59


The code in the cell below finds a linear models SocialMedia engagement as a function of TimePlaying. Execute this code and examine the results.   

> The code in the cell below uses the R style model formula. This modeling language was introduced in Chambers and Hastie, 1992, Statistical Models in S.     

> For a good [**cheatsheet and summary of the R modeling language**](http://faculty.chicagobooth.edu/richard.hahn/teaching/formulanotation.pdf) look at the posting by Richard Hahn of the Chicago Booth School.    

> Models are defined by an equation using the $\sim$ symbol to mean modeled by. In summary, the variable to be modeled is always on the left. The relationship between the variable to be modeled on the right. This basic scheme can be written: 

$$dependent\ variable\sim indepenent\ variables$$

> For example, if the dependent variable (dv) is modeled by two independent variables (var1 and var2), with no interaction, the formula would be:
$$dv \sim var1 + var2$$

In [11]:
social_time_model = smf.ols(formula = 'SocialMedia ~ TimePlaying', data=effect_data).fit()
social_time_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            SocialMedia   R-squared:                       0.268
Model:                            OLS   Adj. R-squared:                  0.266
Method:                 Least Squares   F-statistic:                     182.1
Date:                Mon, 06 Jan 2020   Prob (F-statistic):           1.35e-35
Time:                        18:10:21   Log-Likelihood:                -627.90
No. Observations:                 500   AIC:                             1260.
Df Residuals:                     498   BIC:                             1268.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       1.9860      0.038     52.174      0.000       1.911       2.061
TimePlaying     2.0973      0.155     13.496      0.000       1.792       2.403
==============================================================================
Omnibus:                        4.918   Durbin-Watson:                   1.975
Prob(Omnibus):                  0.086   Jarque-Bera (JB):                4.957
Skew:                           0.221   Prob(JB):                       0.0839
Kurtosis:                       2.795   Cond. No.                         4.08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The summary of the ordinary least squares model contains quite a lot information. We will only focus on a few of these values:    
- The value of the coefficient is an estimate of the **effect size**. In this case the effect being measured is TimePlaying. 
- The confidence interval of the coefficient value which indicates the range of likely values for the effect size. It is important to keep in mind that any estimated effect size is uncertain and not exact. 
- The adjusted R-squared value which is the ratio of the ratio of the variance of the residuals of from the fitted model and the variance of the dependent variable. 



> #### Summary of R-Squared

> - **R squared or $R^2$**, also known as the **coefficient of determination**,  
$$R^2 = 1 - \frac{SS_{res}}{SS_{tot}}$$  
where,   
$SS_{res} = \sum_{i=1}^N r_i^2$, or the sum of the squared residuals,   
$SS_{tot} = \sum_{i=1}^N y_i^2$, or the sum of the squared label values.  

> In other words, $R^2$ is  measure of the reduction in sum of squared values between the raw label values and the residuals. If the model has not reduced the sum of squares of the labels (a useless model!), $R^2 = 0$. On the other hand, if the model fits the data perfectly so all $r_i = 0$, then $R^2 = 1$. 

> - **Adjusted R squared or $R^2_{adj}$** is $R^2$ adjusted for degrees of freedom in the model,
$$R^2_{adj} = 1 - \frac{var(r)}{var(y)} = 1 - \frac{\frac{SS_{res}}{(n - p -1)}}{\frac{SS_{tot}}{(n-1)}}$$  
where,   
$var(r) = $ the variance of the residuals,   
$var(y) = $ the variance of the labels,
$n = $ the number of samples or cases,
$p = $ number of model parameters.  

> The interpretation of $R^2_{adj}$ is the same as $R^2$. In many cases there will be little difference. However if the number of parameters is significant with respect to the number of cases, $R^2$ will give an overly optimistic measure of model performance. In general, the difference between $R^2_{adj}$ and $R^2$ becomes less significant as the number of cases $n$ grows. However, even for 'big data' models there can be a significant difference if there are a large number of model parameters.   


As a next step in the analysis we will add another variable to the model, Fan. Execute the code in the cell below and examine the summary of the new model. 

In [12]:
social_time_fan_model = smf.ols(formula='SocialMedia ~ TimePlaying + Fan', data=effect_data).fit()
social_time_fan_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            SocialMedia   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.491
Method:                 Least Squares   F-statistic:                     241.4
Date:                Mon, 06 Jan 2020   Prob (F-statistic):           5.47e-74
Time:                        18:10:21   Log-Likelihood:                -536.11
No. Observations:                 500   AIC:                             1078.
Df Residuals:                     497   BIC:                             1091.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       1.9862      0.032     62.628      0.000       1.924       2.048
TimePlaying     0.5334      0.167      3.196      0.001       0.206       0.861
Fan             0.6136      0.041     14.848      0.000       0.532       0.695
==============================================================================
Omnibus:                        3.601   Durbin-Watson:                   2.021
Prob(Omnibus):                  0.165   Jarque-Bera (JB):                3.569
Skew:                           0.207   Prob(JB):                        0.168
Kurtosis:                       2.983   Cond. No.                         5.34
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Notice two changes between the first model and the model using two variables:
- The coefficient (effect) value for the variable Fan is marginally larger than the effect for TimePlaying. 
- The adjusted R-squared value is larger indicating this model explains more of the variance of the data.    

Another possibility is to model social media engagement by TimePlaying and GameFamiliarity. Execute the code in the cell below, examine the results, and compare them to the previous models. 

In [13]:
social_time_familarity_model = smf.ols(formula='SocialMedia ~ TimePlaying + GameFamiliarity', data=effect_data).fit()
social_time_familarity_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            SocialMedia   R-squared:                       0.465
Model:                            OLS   Adj. R-squared:                  0.463
Method:                 Least Squares   F-statistic:                     216.1
Date:                Mon, 06 Jan 2020   Prob (F-statistic):           2.91e-68
Time:                        18:10:21   Log-Likelihood:                -549.38
No. Observations:                 500   AIC:                             1105.
Df Residuals:                     497   BIC:                             1117.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           1.9859      0.033     60.980      0.000       1.922       2.050
TimePlaying         3.5593      0.171     20.783      0.000       3.223       3.896
GameFamiliarity    -0.5637      0.042    -13.542      0.000      -0.645      -0.482
==============================================================================
Omnibus:                        0.661   Durbin-Watson:                   1.975
Prob(Omnibus):                  0.719   Jarque-Bera (JB):                0.474
Skew:                           0.023   Prob(JB):                        0.789
Kurtosis:                       3.144   Cond. No.                         5.43
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Notice that the sign of the GameFamiliarity effect is now negative. Given these results, we say that GameFamiliarity is a **confounding effect**. In other words, adding GameFamiliarity confounds, or masks the other effects, but is not explanitory. 

Given the above analysis, it seems that Fan is the variable that best explains social media engagement. This idea is easy to test, by creating a linear model of social media engagement as a function of Fan. Execute the code in the cell below to compute the model and display the summary. 

In [14]:
social_fan_model = smf.ols(formula='SocialMedia ~ Fan', data=effect_data).fit()
social_fan_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            SocialMedia   R-squared:                       0.482
Model:                            OLS   Adj. R-squared:                  0.481
Method:                 Least Squares   F-statistic:                     464.1
Date:                Mon, 06 Jan 2020   Prob (F-statistic):           3.17e-73
Time:                        18:10:21   Log-Likelihood:                -541.20
No. Observations:                 500   AIC:                             1086.
Df Residuals:                     498   BIC:                             1095.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.9862      0.032     62.059      0.000       1.923       2.049
Fan            0.6969      0.032     21.543      0.000       0.633       0.760
==============================================================================
Omnibus:                        4.591   Durbin-Watson:                   2.007
Prob(Omnibus):                  0.101   Jarque-Bera (JB):                4.481
Skew:                           0.231   Prob(JB):                        0.106
Kurtosis:                       3.047   Cond. No.                         1.01
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Notice the following points about this model:
- The effect size for Fan is large relative to effect sizes in other models.  
- The R-squared value is nearly as large as the best previous model, indicating that the Fan variable has good explanatory power. 
- The confidence interval of the effect size is relatively narrow.  

From this analysis, what can you conclude about the relationship between Fan and TimePlaying and SocialMedia. 

##### Copyright 2020, Stephen F Elston. All rights reserved. 